# Connecting to the Meta Database

In [1]:
import pandas as pd
from sqlalchemy import create_engine, MetaData 
from sqlalchemy.orm import sessionmaker
from datatree import open_datatree
import matplotlib.pyplot as plt
import yaml
import intake_sql
from pathlib import Path

def connect(URI):
    engine = create_engine(URI)
    metadata_obj = MetaData()
    metadata_obj.reflect(engine)
    return metadata_obj, engine

with Path('db_config.yml').open() as handle:
    config = yaml.load(handle, yaml.Loader)
    
URI = f"postgresql://dbreader:fairly-mast@{config['host']}:{config['port']}/mast_db"
metadata, engine = connect(URI)
Session = sessionmaker(bind = engine)
session = Session()

### Database Stats

In [2]:
num_shots = session.query(metadata.tables['shots']).count()
num_signals = session.query(metadata.tables['signals']).count()

print(f'Number of shots: {num_shots}')
print(f'Number of signals: {num_signals}')

Number of shots: 25556
Number of signals: 1151


### Querying the Metadatabase

Query the shot table and shot the results

In [3]:
# Find shot IDs
query = (
    session.query(metadata.tables['shots'])
)

result = pd.read_sql(query.statement, con=engine.connect())
result

,shot_id,timestamp,reference_shot,scenario,current_range,heating,divertor_config,pellets,plasma_shape,rmp_coil,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,11695,2004-12-13 11:54:00+00:00,NaN,NaN,None,None,Conventional,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
1,11696,2004-12-13 12:07:00+00:00,NaN,NaN,None,None,Conventional,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
2,11697,2004-12-13 12:19:00+00:00,NaN,NaN,None,None,Conventional,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
3,11698,2004-12-13 12:31:00+00:00,NaN,NaN,None,None,Conventional,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
4,11699,2004-12-13 12:45:00+00:00,NaN,NaN,None,None,Conventional,False,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25551,47202,2023-03-31 14:59:33+00:00,45847.0,NaN,None,None,None,False,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
25552,47203,2023-03-31 15:19:43+00:00,NaN,NaN,None,None,None,False,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
25553,47204,2023-03-31 15:38:20+00:00,NaN,NaN,None,None,None,False,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN
25554,47205,2023-03-31 15:52:21+00:00,44539.0,NaN,None,None,None,False,None,False,...,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,NaN


Show unique campaigns in the database

In [4]:
result['campaign'].unique()

array(['M5', 'M6', 'M7', 'M8', 'M9', 'MU1', 'MU2', 'MU3'], dtype=object)

Filtering with CPF Summary Data and shot IDS

A more advanced query. Here we: 
 - Find shots with a given CPF value
 - Find corresponding signals
 - Filter signals by name

In [5]:
shots = metadata.tables['shots']
signals = metadata.tables['signals']
shot_signal_link = metadata.tables['shot_signal_link']

# Query all shots with zmag_efit > .04
qshots = (
    session.query(shots)
          .filter(shots.c.campaign == 'M9')
)
qshots = pd.read_sql(qshots.statement, con=engine.connect())
shot_ids = qshots['shot_id'].values
shot_ids = list(map(str, shot_ids))

# Query for corresponding signal IDs
qshot_signal = (
    session.query(shot_signal_link.c.signal_id)
    .filter(shot_signal_link.c.shot_id.in_(qshots.shot_id))
    .distinct()
)
qshot_signal = pd.read_sql(qshot_signal.statement, con=engine.connect())

# Query for signal data, filter only names containing 'acd'
qsignal = (
    session.query(signals)
    .filter(signals.c.signal_id.in_(qshot_signal.signal_id))
    # .filter(signals.c.name.contains('AMC'))
)

result = pd.read_sql(qsignal.statement, con=engine.connect())
result

,signal_id,name,units,rank,uri,description,signal_type,quality,doi,camera_metadata,camera
0,1,ABM_CALIB_SHOT,,1,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,Calibration Shot,Analysed,Not Checked,,None,None
1,2,ABM_CHANNEL_STATUS,,2,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,channel_status,Analysed,Not Checked,,None,None
2,3,ABM_CHANNEL_TYPE,,2,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,channel_type,Analysed,Not Checked,,None,None
3,4,ABM_GAIN,,2,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,GAIN,Analysed,Not Checked,,None,None
4,5,ABM_I-BOL,W/m^2,2,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,i-bol,Analysed,Not Checked,,None,None
...,...,...,...,...,...,...,...,...,...,...,...
911,1143,ESM_V_LOOP_STATIC,V,1,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,static V_loop,Analysed,Not Checked,,None,None
912,1144,ESM_W_DOT,W,1,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,W dot,Analysed,Not Checked,,None,None
913,1145,ESM_X,W,1,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,"""stored magnetic power""",Analysed,Not Checked,,None,None
914,1148,ESX_PASSNUMBER,Passno,1,/home/ir-jack5/rds/rds-ukaea-mast-sPGbyCAPsJI/...,Passno,Analysed,Not Checked,,None,None
